### Feature Engineering

In [1]:
import yfinance as yf
import pandas as pd
import glob
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# file_paths = glob.glob('./data/*.csv')

# dfs = {}

# for file_path in file_paths:
#     file_name = file_path.split('/')[-1].replace('.csv', '')
#     dfs[file_name] = pd.read_csv(file_path)


In [6]:
file_dir = './data/'

['SOXX', 'QQQ', 'XLF', 'CARZ', 'VOX', 'PBJ', 'PJP', 'XLE', 'XLU', 'ITA']

# read data
SP500 = pd.read_csv(file_dir + 'SP500.csv')
NASDAQ = pd.read_csv(file_dir + 'NASDAQ.csv')
SOXX = pd.read_csv(file_dir + 'SOXX.csv')
QQQ = pd.read_csv(file_dir + 'QQQ.csv')
XLF = pd.read_csv(file_dir + 'XLF.csv')
CARZ = pd.read_csv(file_dir + 'CARZ.csv')
VOX = pd.read_csv(file_dir + 'VOX.csv')
PBJ = pd.read_csv(file_dir + 'PBJ.csv')
PJP = pd.read_csv(file_dir + 'PJP.csv')
XLE = pd.read_csv(file_dir + 'XLE.csv')
XLU = pd.read_csv(file_dir + 'XLU.csv')
ITA = pd.read_csv(file_dir + 'ITA.csv')


In [7]:
def cal_return(buy_price, sell_price, dividend=0):
    """
    주식 수익률 계산 함수

    :param buy_price: 주식 구매가
    :param sell_price: 주식 판매가
    :param dividend: 받은 배당금 (기본값은 0)
    :return: 수익률 (%)
    """
    # 수익률 계산
    return_rate = ((sell_price + dividend - buy_price) / buy_price) * 100
    
    return return_rate


In [8]:
# feature engineering function
def feature_engineering(tick, bm, n_days):
    # 날짜
    date = tick['Date']

    # 수익률 계산
    #price = tick['Adj Close'].diff(n_days) - bm['Adj Close'].diff(n_days) # actual return
    price = tick['Adj Close'].pct_change(n_days) - bm['Adj Close'].pct_change(n_days) # return(%)
    volume = tick['Volume']
    high_low_gap = tick['High'] - tick['Low']
    
    # EMA(이동평균선) 계산
    price_ema_5 = tick['Adj Close'].rolling(window=5).mean()
    price_ema_10 = tick['Adj Close'].rolling(window=10).mean()
    price_ema_20 = tick['Adj Close'].rolling(window=20).mean()
    price_ema_60 = tick['Adj Close'].rolling(window=60).mean()

    # 변동성 지표 계산
    vol_ema_5 = tick['Adj Close'].rolling(window=5).std()
    vol_ema_10 = tick['Adj Close'].rolling(window=10).std()
    vol_ema_20 = tick['Adj Close'].rolling(window=20).std()
    vol_ema_60 = tick['Adj Close'].rolling(window=60).std()

    # 계산된 특성을 데이터프레임으로 모으기
    features = pd.DataFrame({
        'date': date,
        'price': price,
        'volume': volume,
        'high_low_gap': high_low_gap,
        'price_ema_5': price_ema_5,
        'price_ema_10': price_ema_10,
        'price_ema_20': price_ema_20,
        'price_ema_60': price_ema_60,
        'vol_ema_5': vol_ema_5,
        'vol_ema_10': vol_ema_10,
        'vol_ema_20': vol_ema_20,
        'vol_ema_60': vol_ema_60
    })
    
    return features


In [9]:
SOXX_new = feature_engineering(SOXX, SP500, 1)
SOXX_new

,date,price,volume,high_low_gap,price_ema_5,price_ema_10,price_ema_20,price_ema_60,vol_ema_5,vol_ema_10,vol_ema_20,vol_ema_60
0,2004-01-02,NaN,74000,0.980003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-01-05,0.020144,103300,1.389999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004-01-06,0.009469,170100,1.279999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004-01-07,0.006606,118300,1.139999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2004-01-08,0.024883,208600,1.769997,54.729464,NaN,NaN,NaN,1.612719,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5028,2023-12-22,0.001544,785900,7.469971,565.404004,559.324664,535.904190,496.878355,7.184150,12.025646,25.667777,36.583558
5029,2023-12-26,0.013438,635500,9.760010,568.069421,563.460846,539.216827,498.665721,9.763365,11.188771,26.898056,37.976400
5030,2023-12-27,0.001139,625800,5.359985,570.428015,567.357794,542.754167,500.438164,11.510843,9.765425,27.593824,39.316139
5031,2023-12-28,-0.002245,351000,4.309998,575.946008,570.310382,545.960051,502.356285,6.414749,8.609547,28.072021,40.364843
